<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Generating the music with transformer

In [3]:
pip install tensorflow

In [4]:
%load_ext autoreload
%autoreload 2

import os
import glob
import numpy as np
import time
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models , losses , callbacks

import music21

from tensorflow_utils import(
    parse_midi_files,
    load_parsed_files,
    get_midi_note,
    SinePositionEncoding,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: ignored

# 0. Parameters

In [6]:
PARSE_MIDI_FILES = True
PARSED_DATA_PATH = "/app/notebooks/11_music/01_transformer/parsed_data/"
DATASET_REPETITIONS = 1

SEQ_LEN = 50
EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 5
DROPOUT_RATE = 0.3
FEED_FORWARD_DIM = 256
LOAD_MODEL = False

#optimization
EPOCHS = 50
BATCH_SIZE = 256

GENERATE_LEN = 50

# 1. Prepare the Data

In [1]:
#Load the data
file_list = glob.glob("/app/data/bach-cello/*.mid")
print(f"Found {len(file_list)} midi files")

NameError: ignored

In [2]:
parser = music21.converter
example_score = (
    music21.converter.parse(file_list[1]).splitAtQuarterLength(12)[0].chordify()
)


NameError: ignored

In [3]:
example_score.show()


NameError: ignored

In [4]:
example_score.show("text")

NameError: ignored

In [5]:
if PARSE_MIDI_FILES:
  notes, durations = parse_midi_files(
      file_list, parser, SEQ_LEN + 1 , PARSED_DATA_PATH
  )
else:
  notes, durations = load_parsed_files()

NameError: ignored

In [6]:
example_notes = notes[658]
example_durations = durations[658]
print("\nNotes string\n" , example_notes, "...")
print("\nDuration string\n" , example_durations, "...")


NameError: ignored

#2. Tokenize the data


In [7]:
def create_dataset(elements):
  ds = (
      tf.data.Dataset.from_tensor_slices(elements)
      .batch(BATCH_SIZE , drop_reminder = True)
      .shuffle(1000)
  )
  vectorize_layer = layers.TextVectorization(
      standardize = None, output_mode = "int"
  )
  vectorize_layer.adapt(ds)
  vocab = vectorize_layer.get_vocabulary()
  return ds, vectorize_layer, vocab


notes_seq_ds, notes_vectorize_layer, notes_vocab = create_dataset(notes)
duratoins_seq_ds , durations_vectorize_layer, durations_vocab = create_dataset(
    durations
)
seq_ds = tf.data.Dataset.zip((notes_seq_ds , durations_seq_ds))


NameError: ignored

In [8]:
#Display the same example notes and durations converted to ints
example_tokenised_notes = notes_vectorize_layer(example_notes)
example_tokenised_durations = durations_vectorize_layer(example_durations)
print("{:10} {:10}" .format("note token" , "duration token"))
for i , (note_int, duration_int) in enumerate(
    zip(
        example_tokenised_notes.numpy()[:11],
        example_tokenised_durations.numpy()[:11]
    )
):
    print(f"{note_int:10}{duration_int:10}")


NameError: ignored

In [9]:
notes_vocab_size = len(notes_vocab)
durations_vocab_size = len(durations_vocab)

#Display some token:note mappings
print(f"\nNOTES_VOCAB: length = {len(notes_vocab)}")
for i , note in enumerate(notes_vocab[:10]):
  print(f"{i}: {note}")

print(f"\nDURATIONS_VOCAB: length = {len(durations_vocab)}")
#Display some token:duration mappings
for i , note in enumerate(durations_vocab[:10]):
  print(f"{i}: {note}")


NameError: ignored

#3. Create the Training Set

In [11]:
#Create the training set of sequences and the same sequences shifted by one note
def prepare_inputs(notes, durations):
  notes = tf.expand_dims(notes, -1)
  durations = tf.expand_dims(durations, -1)
  tokenized_notes = notes_vectorize_layer(notes)
  tokenized_durations = durations_vectorize_layer(durations)
  x = (tokenized_notes[:, : -1], tokenized_durations[:, :-1])
  y = (tokenized_notes[:, 1:], tokenized_durations[:, 1:])
  return x, y

ds = seq_ds.map(prepare_inputs).repeat(DATASET_REPETITIONS)

NameError: ignored

In [12]:
example_input_output = ds.take(1).get_single_element()
print(example_input_output)

NameError: ignored

#5. Create the causal attention mask funciton

In [1]:
def causal_attention_mask(batch_size, n_dest , n_src , dtype):
  i = tf.range(n_dest)[: , None]
  j = tf.range(n_src)
  m = i >= j - n_src + n_dest
  mask = tf.cast(m, dtype)
  mask = tf.reshape(mask, [1, n_dest , n_src])
  mult = tf.concat(
      [tf.expand_dims(batch_size , -1) , tf.constant([1,1], dtype = tf.int32)], 0
  )
  return tf.tile(mask, mult)

np.transpose(causal_attention_mask(1, 10 ,10, dtype=tf.int32)[0])



NameError: ignored

#6. Create a Transformer Block Layer

In [ ]:
class TransformerBlock(layers.Layer):
  def __init__(
    self,
    num_heads,
    key_dim,
    embed_dim,
    ff_dim,
    name,
    dropout_rate = DROPOUT_RATE,
  ):
    super(TransformerBlock, self).__init__(name = name)
    self.num_heads = num_heads
    self.key_dim = key_dim
    self.embed_dim = embed_dim
    self.ff_dim = ff_dim
    self.dropout_rate = dropout_rate
    self.attn = layers.MultiHeadAttention(
        num_heads, key_dim , output_shape = embed_dim
    )
    self.dropout_1 = layers.Dropout(self.dropout_rate)
    self.ln_1 = layers.LayerNormalization(epsilon = 1e-6)
    self.ffn_1 = layers.Dense(self.ff_dim, activation = "relu")
    self.ffn_2 = layers.Dense(self.embed_dim)
    self.dropout_2 = layers.Dropout(self.dropout_rate)
    self.ln_2 = layers.LayerNormalization(epsilon = 1e-6)

  def call(self, inputs):
    input_shape = tf.shape(inputs)
    batch_size = input_shape[0]
    seq_len = input_shape[1]
    causal_mask = causal_attention_mask(
        batch_size , seq_len , seq_len , tf.bool
    )
    attention_output , attention_scores = self.attn(
        inputs,
        inputs,
        attention_mask = causal_mask,
        return_attention_scores = True,

    )
    attention_output = self.dropout_1(attention_output)
    out1 = self.ln_1(inputs + attention_output)
    ffn_1 = self.ffn_1(out1)
    ffn_2 = self.ffn_2(ffn_1)
